In [1]:
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import common
import evaluate_solution
import generate_shipping_schedule

In [2]:
def prepare_lp_variables(month,dfs):
    mod_dem_for = dfs['mod_dem_for']
    pro_cost = dfs['pro_cos']
    pro_cap = dfs['pro_cap']
    dem_price = dfs['dem_pri']
    products={}
    regions={}
    for i in range(1,82):
        products['P'+str(i)]={}
    for i in range(1,19):
        regions['R'+str(i)]={}
    lines={};
    lines['A1']={'Plant':'A','Line':1,'Cost':33536}
    lines['A2']={'Plant':'A','Line':2,'Cost':15947}
    lines['B1']={'Plant':'B','Line':1,'Cost':13333}
    lines['B2']={'Plant':'B','Line':2,'Cost':13333}
    lines['B3']={'Plant':'B','Line':3,'Cost':13333}
    lines['C1']={'Plant':'C','Line':1,'Cost':54353}
    # Get the monthly demand per product sku
    mod_dem_for = mod_dem_for[mod_dem_for.Month==month].copy()
#    ppd = mod_dem_for.groupby(['Product_ID']).Demand.sum()
    for product_id in products:
        products[product_id]['Demand']=(mod_dem_for[mod_dem_for.Product_ID==product_id].
                                        Demand.values[0])
        products[product_id]['Aggregated']=(mod_dem_for[mod_dem_for.Product_ID==product_id].
                                        Aggregated.values[0])
        products[product_id]['LowProfit']=(mod_dem_for[mod_dem_for.Product_ID==product_id].
                                        LowProfit.values[0])        
    # Get the capacity for each line and product
    # Calculate the profit per MT of production at a particular line. For now we are ignoring
    # the shipping cost that depends on region of demand and are taking the max of the price
    # across region
    for line_name in lines.keys():
        line = lines[line_name]
        plant_name = line['Plant']
        line_id = line['Line']
        line['Capacity']={}
        line['Profit']={}
        for product_id in products.keys():
            line['Capacity'][product_id]=(pro_cap[(pro_cap.Plant==plant_name)&
                        (pro_cap.Line==line_id)&(pro_cap.Product==product_id)]).Capacity.values[0]
            line['Profit'][product_id] = common.get_profit(product_id,line_name,month,dfs)
            if line['Profit'][product_id]<0:
                line['Profit'][product_id]=0
    return products,lines

def prepare_test_variables():
    products={'P1':{'Demand':300},'P2':{'Demand':300},'P3':{'Demand':0}}
    lines={'A1':{'Plant':'A','Line':'1','Capacity':{'P1':5,'P2':8,'P3':5},
                 'Profit':{'P1':19,'P2':15,'P3':5}},
           'B2':{'Plant':'B','Line':'2','Capacity':{'P1':5,'P2':8,'P3':0},
                 'Profit':{'P1':21,'P2':17,'P3':5}}}
    return products,lines

In [3]:
def calculate_objective_summary(unknowns,products,lines):
    profit=0
    total_demand=0
    satisfied_demand=0
    usage={}
    for product_id in sorted(products.keys()):
        total_demand += products[product_id]['Demand']
    for unknown_id in sorted(unknowns.keys()):
        unknown = unknowns[unknown_id]
        uname = unknown.name()
        quantity = unknown.solution_value()
        product_id,line_id = uname.split('-')
        line = lines[line_id]
        product = products[product_id]
        profit += (line['Profit'][product_id]*quantity)
        satisfied_demand += quantity
        if line_id not in usage:
            usage[line_id]=0
        if(quantity > 0):
            usage[line_id] += (quantity/line['Capacity'][product_id])
    summary = {'total_demand':total_demand,'satisfied_demand':satisfied_demand,
              'profit':profit,'usage':usage} 
    return summary

In [4]:
def get_line_orders(unknowns):
    line_orders={'A1':[],'A2':[],'B1':[],'B2':[],'B3':[],'C1':[]}
    for unknown_id in unknowns:
        unknown = unknowns[unknown_id]
        if unknown.solution_value()==0:
            continue
        product_id,line_id = unknown_id.split('-')
        line_orders[line_id].append({'product':product_id,
            'quantity':unknown.solution_value()})
    return line_orders

In [39]:
def solve_optimization_problem(products,lines,month,line_days_config):
    solver = pywraplp.Solver('LinearExample',pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    unknowns={}
    constraints=[]
    #Add the variables of the linear program. These are placeholders for quantity of 
    #product manufactured in each line
    for product_id in sorted(products.keys()):
        product = products[product_id]
        for line_id in sorted(lines.keys()):
            line = lines[line_id]
            prod_lineid_str = product_id+"-"+line_id
            if (line['Capacity'][product_id])>0:
                unknowns[prod_lineid_str] = solver.NumVar(0,float(28*(line['Capacity'][product_id])),
                                                      prod_lineid_str)

    # Add the constraints to ensure every line can operate only for 30 days in a month
    for line_id in sorted(lines.keys()):
        line = lines[line_id]
        constraint = solver.Constraint(15,line_days_config[month][line_id])
        for product_id in sorted(products.keys()):
            prod_lineid_str = product_id+"-"+line_id
            if line['Capacity'][product_id] > 0:
                constraint.SetCoefficient(unknowns[prod_lineid_str],(1/line['Capacity'][product_id]))
        constraints.append(constraint)

    # Add the constraint to limit the amount produced to monthly demand
    for product_id in sorted(products.keys()):
        product = products[product_id]
        low_limit = product['Demand']*0.3
        high_limit = product['Demand']*1
        if product['Aggregated']==1:
            low_limit = product['Demand']
        if product['LowProfit']:
            high_limit = product['Demand']*0.75
        constraint = solver.Constraint(low_limit,high_limit)
        
        for line_id in sorted(lines.keys()):
            line = lines[line_id]
            prod_lineid_str = product_id+"-"+line_id
            if line['Capacity'][product_id] > 0:
                constraint.SetCoefficient(unknowns[prod_lineid_str],1)
        constraints.append(constraint)
        
    #Objective
    objective = solver.Objective()
    for product_id in sorted(products.keys()):
        product = products[product_id]    
        for line_id in sorted(lines.keys()):
            line = lines[line_id]
            prod_lineid_str = product_id+"-"+line_id
            if prod_lineid_str in unknowns:
                objective.SetCoefficient(unknowns[prod_lineid_str],line['Profit'][product_id])
    objective.SetMaximization()
    solver.SetTimeLimit(100000)
    #print('Number of variables =', solver.NumVariables())
    #print('Number of constraints =', solver.NumConstraints())
    status = solver.Solve()
    print("**** Solver status is ***",status)
    objective_summary = calculate_objective_summary(unknowns,products,lines)
    line_orders = get_line_orders(unknowns)
    return line_orders,objective_summary

In [6]:
def generate_work_allocation(line_id,line_orders,last_allocation,products,lines,dfs):
    cha_day = dfs['cha_day']
    free_days=0
    my_line_orders = line_orders[line_id]
    for item in my_line_orders:
        product = item['product']
        quantity = item['quantity']
        item['profit']=quantity*lines[line_id]['Profit'][product]
        item['no_of_days'] = max(np.floor(quantity/lines[line_id]['Capacity'][product]),1)
        item['capacity'] = lines[line_id]['Capacity'][product]
    my_line_orders.sort(key=lambda x:x['profit'],reverse=True)
    #print(my_line_orders)
    #now start allocating them one by one till we run out of days
    days_remaining = 30
    order_index = 0
    allocation=[]
    #account for change days due to product switch at beginning of month
    if last_allocation != None:
        no_of_initial_cha_days = (cha_day[(cha_day.From==last_allocation['product'])&
            (cha_day.To==my_line_orders[0]['product'])].Days.values[0]-last_allocation['days_remaining'])
        if no_of_initial_cha_days == 0:
            no_of_initial_cha_days=1
        days_remaining -= no_of_initial_cha_days
    while days_remaining > 0:
        #allocate the current job
        if order_index >= len(my_line_orders):
            break
        order = my_line_orders[order_index]
        if order['no_of_days'] < days_remaining:
            if order['no_of_days'] < 14:
                order['allocated_days']=order['no_of_days']
                order['fulfilled_quantity'] = min(order['quantity'],
                                                  order['capacity']*order['allocated_days'])
                days_remaining -= order['allocated_days']
                order['days_remaining'] = days_remaining
                order_index += 1
                allocation.append(order)
            else:
#                print('More than 14 day allocation',order)
                order['allocated_days']=14
                order['fulfilled_quantity'] = min(order['quantity'],
                                                  order['capacity']*order['allocated_days'])
                days_remaining -= order['allocated_days']
                order['days_remaining'] = days_remaining
                order_index += 1
                allocation.append(order)                
        else:
            if days_remaining < 14:
                order['allocated_days']=days_remaining
                order['fulfilled_quantity']=order['capacity']*order['allocated_days']
                days_remaining=0
                order['days_remaining']=0
                order_index +=1 
                allocation.append(order)
            else:
                #TODO The 14 problem is being circumvent here. Fix it.
#                print('More than 14 day allocation',order)
                order['allocated_days']=14
                order['fulfilled_quantity']=order['capacity']*order['allocated_days']
                days_remaining-=14
                order['days_remaining']=days_remaining
                order_index +=1 
                allocation.append(order)
                break
        if order_index >= len(my_line_orders):
            #TODO We can run work from other queue
            free_days = days_remaining
            break
        # TODO Add a additional logic to consider the crossover time as part of the cost logic
        # and choosing the next product
        # Add changeover days
        days_remaining -= cha_day[(cha_day.From==order['product'])&
                                  (cha_day.To==my_line_orders[order_index]['product'])].Days.values[0]
    return(allocation,free_days)

In [16]:
def generate_manufacturing_sequence_file(allocations,dfs):
    lines=['A1','A2','B1','B2','B3','C1']
    cha_day = dfs['cha_day']
    man_seq=[]
    for line_id in lines:
        plant = line_id[0]
        line_no = line_id[1]
        prev_product=None
        for month in sorted(allocations.keys()):
            monthly_allocation = allocations[month][line_id]
            day = 1
            if prev_product != None: #check to ensure we leave enough cross over days
                index = -1
                while True:
                    if len(man_seq[index])==4 or man_seq[index][4]==None: #if blank line
                        index -=1
                    else:
                        break
                blank_days = abs(index)-1
                first_item = monthly_allocation[0]
                cds = cha_day[(cha_day.From==prev_product['product']) &
                                (cha_day.To==first_item['product'])
                                 ].Days.values[0] - prev_product['days_remaining']
                if cds==0:
                    cds=1  #If it is the same product, then leave a day. Its easier TODO
                while blank_days < cds:
                    man_seq.append([plant,line_no,month,day,])
                    day += 1
                    blank_days += 1

            for index,item in enumerate(monthly_allocation):
                for ad in range(int(item['allocated_days'])):
                    prev_product = item
                    man_seq.append([plant,line_no,month,day,item['product']])
                    day += 1
                    if day > 30:
                        break
                if day>30:
                    break
                if index+1 <len(monthly_allocation):
                    cds = cha_day[(cha_day.From==item['product']) &
                                (cha_day.To==monthly_allocation[index+1]['product'])
                                 ].Days.values[0]
                    for cd in range(cds):
                        man_seq.append([plant,line_no,month,day,])
                        day += 1
                        if day > 30:
                            break
                    if day>30:
                        break
                else: # nothing more to allocate
                    break
            while day <= 30:
                man_seq.append([plant,line_no,month,day,''])
                day += 1
    msdf = pd.DataFrame(man_seq)
    msdf.columns=['Plant','Line','Month','Day','Product_ID']
    msdf.to_csv('manufacture_sequence.csv',index=False)

In [8]:
def check_constraints(dfs):
    #Check the 14 day constraint
    man_seq = pd.read_csv('manufacture_sequence.csv')
    curr_prod_id=None
    curr_count=0
    for index,row in man_seq.iterrows():
        if row.Product_ID != curr_prod_id:
            curr_prod_id=row.Product_ID
            curr_count=1
        else:
            curr_count +=1
            if curr_count > 14:
                print("CONSTRAINT VIOLATED - MORE THAN 14 DAYS")
                print(row)
    #Check quarterly 30% constraint
    missed_constraints=[]
    dem_for = pd.read_csv('demand_forecast.csv')
    prod_cap = pd.read_csv('production_capacity.csv')
    products=[]
    for i in range(1,82):
        products.append('P'+str(i))
    product_stats={}
    for product in products:
        product_stats[product]={'Total_Demand':0,'Man_Quantity':0}
        product_stats[product]['Total_Demand'] = np.sum(dem_for[(dem_for.Product_ID==product)].Demand)

    for index,row in man_seq.iterrows():
        if not (prod_cap[(prod_cap.Plant==row.Plant) &
                (prod_cap.Line==row.Line)&(prod_cap.Product==row.Product_ID)].Capacity).empty:
            product_stats[row.Product_ID]['Man_Quantity']+= prod_cap[(prod_cap.Plant==row.Plant) &
                (prod_cap.Line==row.Line)&(prod_cap.Product==row.Product_ID)].Capacity.values[0]
    
    for product in products:
        if product_stats[product]['Total_Demand']>0:
            if product_stats[product]['Man_Quantity']/product_stats[product]['Total_Demand'] < 0.3:
                missed_constraints.append({'product':product,
                    'required':product_stats[product]['Total_Demand']*0.3,
                    'manufactured':product_stats[product]['Man_Quantity']})
    return missed_constraints

In [9]:
def check_order_that_missed_allocations(line_orders,allocations):
    missed_orders={}
    for month in range(37,40):
        missed_orders[month]={}
        for line_id in ['A1','A2','B1','B2','B3','C1']:
            missed_orders[month][line_id]=[]
            line_order_products = [x['product'] for x in line_orders[month][line_id] ]
            allocation_products = [x['product'] for x in allocations[month][line_id] ]
            difference = set(line_order_products)-set(allocation_products)
            if len(difference) > 0:
                missed_orders[month][line_id]=list(difference)
    return missed_orders

In [19]:
# Distance callback
class CreateDistanceCallback(object):
    """Create callback to calculate distances between points."""
    def __init__(self,product_list,dfs):
        self.cha_day = dfs['cha_day']
        self.product_list = product_list
    def Distance(self, from_product, to_product):
        from_product_name = self.product_list[from_product]['product']
        to_product_name = self.product_list[to_product]['product']
        to_product_benefit = self.product_list[to_product]['profit']
        change_days = self.cha_day[(self.cha_day.From==from_product_name) & 
                        (self.cha_day.To==to_product_name)].Days.values[0]
        #Here 10000 doesnt make any difference since changeover cost is same for all products
        #on a line
        return (change_days*10000/to_product_benefit) 

def find_depot(products,last_allocation,dfs):
    cha_day = dfs['cha_day']
    if last_allocation==None:
        return 0 #Return the first index if there is no previous allocation
    days_costs = []
    last_allocated_product = last_allocation['product']
    empty_days_from_prev_month = last_allocation['days_remaining']
    for product in products:
        days_cost = cha_day[(cha_day.From==last_allocated_product) & 
                        (cha_day.To==product)].Days.values[0]
        if(days_cost ==0):
            days_cost=1   #TODO For simplicity for now take a day off if old and new is same
        days_costs.append(days_cost-empty_days_from_prev_month)
    return int(np.argmin(days_costs))

        
    
    
def find_optimal_route(product_list,last_allocation,dfs):
    products = [x['product'] for x in product_list]
    tsp_size = len(products)
    num_routes = 1
    depot=find_depot(products,last_allocation,dfs)
    return_route=[]
    if tsp_size > 0:
        routing = pywrapcp.RoutingModel(tsp_size, num_routes, depot)
        search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
        dist_between_nodes = CreateDistanceCallback(product_list,dfs)
        dist_callback = dist_between_nodes.Distance
        routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)
        # Solve, returns a solution if any.
        assignment = routing.SolveWithParameters(search_parameters)
        if assignment:
            # print("Total distance: ",assignment.ObjectiveValue())
            # Inspect solution.
            # Only one route here; otherwise iterate from 0 to routing.vehicles() - 1
            route_number = 0
            index = routing.Start(route_number) # Index of the variable for the starting node.
            route = ''
            while not routing.IsEnd(index):
                # Convert variable indices to node indices in the displayed route.
                return_route.append(products[routing.IndexToNode(index)])
                route += str(products[routing.IndexToNode(index)]) + ' -> '
                index = assignment.Value(routing.NextVar(index))
            route += str(products[routing.IndexToNode(index)])
            #print("Route:\n\n" + route)
        else:
            print('No solution found.')
            print(product_list)
    else:
        print('Specify an instance greater than 0.')
    return return_route

In [11]:
def reorder_line_orders(line_orders,previous_allocations,month,dfs):
    new_line_orders={}
    for line_name in line_orders:
        last_allocation=None
        if previous_allocations != None: 
            last_allocation = previous_allocations[line_name][-1]
        ind_line_order = line_orders[line_name]
        for item in ind_line_order:
            profit = common.get_profit(item['product'],line_name,month,dfs)
            if profit < 1:
                profit=1
            item['profit']=profit
        product_list = [{'product':x['product'],'profit':x['profit']} for x in ind_line_order]
        new_product_ids = find_optimal_route(product_list,last_allocation,dfs)
        new_ind_line_order=[]
        for product_id in new_product_ids:
            new_ind_line_order.append(list(filter(lambda x:x['product']==product_id,ind_line_order))[0])
        new_line_orders[line_name]=new_ind_line_order
    return new_line_orders

In [12]:
def print_overall_progress_status(line_days_config,
        objective_summaries,free_days,missed_orders,missed_constraints):
    for month in range(37,40):
        print("{} - {} / {}".format(month,objective_summaries[month]['satisfied_demand'],
             objective_summaries[month]['total_demand']))
        print('Days configured: ', end=' ')
        for line in line_days_config[month]:
            print("{}-{} ".format(line,line_days_config[month][line]),end=' ')
        print(' ')
        print("Free Days: ",end=' ')
        for line in free_days[month]:
            if free_days[month][line] > 0:
                print("{}-{}".format(line,free_days[month][line]),end=' ')
        print(' ')
        print('Missed orders: ',end=' ')
        for line in missed_orders[month]:
            if len(missed_orders[month][line]) > 0:
                print("{}-{}-{}".format(line,len(missed_orders[month][line]),
                                       missed_orders[month][line]),end=' ')
        print(' ')
    print('Missed constraints- ',len(missed_constraints))
    for constraint in missed_constraints:
        print('{} - {} / {}'.format(constraint['product'],
                    constraint['manufactured'],round(constraint['required'])))


In [13]:
def update_line_days_config(line_days_config,free_days,missed_orders,missed_constraints):
    lines_to_reduce=set()
    for missed_constraint in missed_constraints:
        product = missed_constraint['product']
        for month in missed_orders:
            for line in missed_orders[month]:
                if product in missed_orders[month][line]:
                    lines_to_reduce.add("{}-{}".format(month,line))
    for line_to_reduce in lines_to_reduce:
        month = int(line_to_reduce.split('-')[0])
        line = line_to_reduce.split('-')[1]
        line_days_config[month][line] -= 1
    for month in free_days:
        for line in free_days[month]:
            if free_days[month][line] > 1.5:
                line_days_config[month][line] += 1
    return len(lines_to_reduce) > 0

In [76]:
line_days_config={}
for month in range(37,40):
    line_days_config[month]={}
    for line in ['A1','A2','B1','B2','B3','C1']:
        line_days_config[month][line]=22

In [82]:
dfs = common.read_csvs()

#products,lines = prepare_test_variables()
run_no = 0
#while True:
raw_line_orders={}
refined_line_orders={} #Line orders after reordering
allocations={}
free_days={}
objective_summaries={}
for month in range(37,40):
    free_days[month]={}
    products,lines = prepare_lp_variables(month,dfs)
    line_orders,objective_summary = solve_optimization_problem(products,lines,
                                        month,line_days_config)
    #print(month,line_orders)
    objective_summaries[month]=objective_summary
    raw_line_orders[month]=line_orders
    if month > 37:
        previous_month_allocations = allocations[month-1]
    else:
        previous_month_allocations=None
    line_orders = reorder_line_orders(line_orders,previous_month_allocations,month,dfs)
    refined_line_orders[month]=line_orders
    allocations[month]={}
    for line_name in line_orders:
        last_allocation=None
        if month > 37: 
            last_allocation = allocations[month-1][line_name][-1]
        allocations[month][line_name],line_free_days=generate_work_allocation(line_name,
                        line_orders,last_allocation,products,lines,dfs)
        free_days[month][line_name]=line_free_days
missed_orders = check_order_that_missed_allocations(refined_line_orders,allocations)
generate_manufacturing_sequence_file(allocations,dfs)
missed_constraints = check_constraints(dfs)
print_overall_progress_status(line_days_config,objective_summaries,free_days,missed_orders,missed_constraints)
#     generate_shipping_schedule.generate_shipping_schedule(dfs)
#     print('Margin',evaluate_solution.get_margin_percent())
#     if update_line_days_config(line_days_config,free_days,missed_orders,missed_constraints)==False:
#         break
#     run_no +=1 
#     print("====================Finished run: ",run_no)
# print("Finished finding right manufacturing sequence.")

**** Solver status is *** 0
**** Solver status is *** 0
**** Solver status is *** 0
37 - 20381.594769600215 / 20381.594769600215
Days configured:  A1-18  A2-22  B1-22  B2-22  B3-22  C1-22   
Free Days:  A2-1.0 B1-14.0 B2-3.0 C1-5.0  
Missed orders:  A1-1-['P3']  
38 - 20090.407838632902 / 20090.407838632906
Days configured:  A1-18  A2-22  B1-22  B2-22  B3-22  C1-22   
Free Days:  A2-2.0 B1-27.0 B2-3.0 C1-9.0  
Missed orders:  A1-1-['P3']  
39 - 20231.78792319211 / 20231.78792319211
Days configured:  A1-22  A2-22  B1-22  B2-22  B3-22  C1-22   
Free Days:  A2-7.0 B1-36.0 B2-1.0 C1-13.0  
Missed orders:  A1-1-['P8'] B3-1-['P4']  
Missed constraints-  0


In [81]:
line_days_config = {
 37: {'A1': 18, 'A2': 22, 'B1': 22, 'B2': 22, 'B3': 22, 'C1': 22},
 38: {'A1': 18, 'A2': 22, 'B1': 22, 'B2': 22, 'B3': 22, 'C1': 22},
 39: {'A1': 22, 'A2': 22, 'B1': 22, 'B2': 22, 'B3': 22, 'C1': 22}}

In [66]:
def track_product(product,line_orders,allocations):
    for month in range(37,40):
        for line in ['A1','A2','B1','B2','B3','C1']:
            this_line_orders = line_orders[month][line]
            this_line_allocations = allocations[month][line]
            for order in this_line_orders:
                if order['product']==product:
                    print("Order: ",month,line,order['quantity'])
            for allocation in this_line_allocations:
                if allocation['product']==product:
                    print("Alloc: ",month,line,allocation['quantity'])
track_product('P62',refined_line_orders,allocations)  

Order:  37 B3 145.18081182514882


In [78]:
line_days_config


{37: {'A1': 22, 'A2': 22, 'B1': 22, 'B2': 22, 'B3': 22, 'C1': 22},
 38: {'A1': 22, 'A2': 22, 'B1': 22, 'B2': 22, 'B3': 22, 'C1': 22},
 39: {'A1': 22, 'A2': 22, 'B1': 22, 'B2': 22, 'B3': 22, 'C1': 22}}